In [1]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [2]:
df=pd.read_csv("prediction_with_batch_nationality.csv")

In [3]:
def tokenize(sentence, level='char'):
    if level == 'char':
        return list(sentence)  # Character-level tokenization
    elif level == 'word':
        return sentence.split()  # Word-level tokenization
    else:
        raise ValueError("Invalid level. Use 'char' or 'word'.")

In [4]:
df['predicted'][0][5:-6]

' چون که هیچ نمی توان بدون بدون دوست صمیمی زندگی . '

In [5]:
df['true_tokens'] = df['correct_sentence'].apply(lambda x: tokenize(x, level='word'))
df['pred_tokens'] = df['predicted'].apply(lambda x: tokenize(x[5:-6], level='word')) #deleting <sos> and <eos>

In [6]:
df.head()

,wrong_sentence,correct_sentence,predicted,true_tokens,pred_tokens
0,[Other] چون که هیچ کس نمی توان بدون دوست صمیمی...,چون که هیچ کس نمی تواند بدون دوست صمیمی زندگی ...,<sos> چون که هیچ نمی توان بدون بدون دوست صمیمی...,"[چون, که, هیچ, کس, نمی, تواند, بدون, دوست, صمی...","[چون, که, هیچ, نمی, توان, بدون, بدون, دوست, صم..."
1,[India] به طور مثال اگر یک صنعت کار چیزهای را ...,به طور مثال اگر یک صنعت کار چیزهای را می سازند...,<sos> به طور مثال اگر یک صنعت کار چیزهای را می...,"[به, طور, مثال, اگر, یک, صنعت, کار, چیزهای, را...","[به, طور, مثال, اگر, یک, صنعت, کار, چیزهای, را..."
2,[Turk] بعضی از این مشکلان ، بودن دور از خانه و...,بعضی از این مشکلان ، بودن دور از خانه واده ، ز...,<sos> بعضی از این مشکلات ، بودن دور از خانواده...,"[بعضی, از, این, مشکلان, ،, بودن, دور, از, خانه...","[بعضی, از, این, مشکلات, ،, بودن, دور, از, خانو..."
3,[Turk] فاصیلی خانه‌ی آقای مهدوی تا معازه‌ از خ...,فاصیلی خانه‌ی آقای مهدوی تا معازه‌ از خود حدود...,<sos> فاصیلی خانه‌ی آقای مهدوی تا معازه‌ از خو...,"[فاصیلی, خانه‌ی, آقای, مهدوی, تا, معازه‌, از, ...","[فاصیلی, خانه‌ی, آقای, مهدوی, تا, معازه‌, از, ..."
4,[Pars] جهان خواران پس از شکست ذلّت بار ب حمایت...,جهان خواران پس از شکست ذلّت بار با حمایت همه ج...,<sos> جهان خواران پس از شکست ذلّت بار ب حمایت ...,"[جهان, خواران, پس, از, شکست, ذلّت, بار, با, حم...","[جهان, خواران, پس, از, شکست, ذلّت, بار, ب, حما..."


In [7]:
df['bleu_score'] = df.apply(lambda row: sentence_bleu([row['true_tokens']], row['pred_tokens']), axis=1)

C:\Users\mahboub\miniforge3\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\mahboub\miniforge3\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\mahboub\miniforge3\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()

In [8]:
average_bleu = df['bleu_score'].mean()
print("Average BLEU Score:", average_bleu)

Average BLEU Score: 0.5842089207605698


In [9]:
df['true_length'] = df['true_tokens'].apply(len)
df['pred_length'] = df['pred_tokens'].apply(len)

inconsistent_lengths = df[df['true_length'] != df['pred_length']]
print(f"Inconsistent lengths found in {len(inconsistent_lengths)} rows.")


Inconsistent lengths found in 543 rows.


In [10]:
def align_sequences(row):
    min_len = min(len(row['true_tokens']), len(row['pred_tokens']))
    row['true_tokens'] = row['true_tokens'][:min_len]
    row['pred_tokens'] = row['pred_tokens'][:min_len]
    return row

df = df.apply(align_sequences, axis=1)

In [11]:
all_true_tokens = sum(df['true_tokens'].tolist(), [])
all_pred_tokens = sum(df['pred_tokens'].tolist(), [])

precision = precision_score(all_true_tokens, all_pred_tokens, average='weighted', zero_division=1)
recall = recall_score(all_true_tokens, all_pred_tokens, average='weighted', zero_division=1)
f1 = f1_score(all_true_tokens, all_pred_tokens, average='weighted', zero_division=1)
accuracy = accuracy_score(all_true_tokens, all_pred_tokens)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)


Precision: 0.7352735094704422
Recall: 0.704715100375539
F1 Score: 0.6937634792157964
Accuracy: 0.704715100375539
